In [ ]:
import pandas as pd

In [ ]:
df_1 = pd.read_csv('bestpoolshop_chlorine.csv')
df_1.head()

In [ ]:
df_2 = pd.read_csv('chlorine_data.csv')
df_2.head()

In [ ]:
df_3 = pd.read_csv('epoolsupply_chlorine.csv')
df_3.head()

In [ ]:
df_4 = pd.read_csv('poolweb_chlorine.csv')
df_4.head()

In [ ]:
df_2.columns

In [ ]:
df_1.head()

In [ ]:
def restructure_df(df):
    """
    Keeps only ['title', 'overview', 'price', 'stock', 'description', 'link'] columns.
    Merges any additional columns into 'description' with labels.

    Parameters:
        df (pd.DataFrame): The input DataFrame

    Returns:
        pd.DataFrame: Modified DataFrame with merged description
    """
    
    # Core columns to retain
    final_cols = ['title', 'overview', 'price', 'stock', 'description', 'link']
    
    # Identify extra columns
    extra_cols = [col for col in df.columns if col not in final_cols]

    # Merge extra columns into description
    df['description'] = df.apply(
        lambda row: f"{row['description']} " + 
                    " ".join([f"{col}: {row[col]}" for col in extra_cols if pd.notna(row[col])]), 
        axis=1
    )

    # Keep only final columns
    return df[final_cols]


In [ ]:
df_1_new = restructure_df(df_1)
df_2_new = restructure_df(df_2)
df_3_new = restructure_df(df_3)
df_4_new = restructure_df(df_4)

In [ ]:
df_1_new.to_csv('bestpoolshop_merged.csv', index=False)
df_2_new.to_csv('freshpool_merged.csv', index=False)
df_3_new.to_csv('epoolsupply_merged.csv', index=False)
df_4_new.to_csv('poolweb_merged.csv', index=False)

In [ ]:
df_4_new.head()

In [ ]:
df_1_new['description'][0]

In [ ]:
def remove_dollar(price):
    """
    Removes $ symbol, commas, extra spaces from price and returns as float.
    Returns 0.0 if conversion fails.
    """
    try:
        price = str(price).replace("$", "").replace(",", "").strip()
        return float(price)
    except:
        return 0.0


In [ ]:
df_1_new.loc[:, 'price'] = df_1_new['price'].apply(remove_dollar)
df_2_new.loc[:, 'price'] = df_2_new['price'].apply(remove_dollar)
df_3_new.loc[:, 'price'] = df_3_new['price'].apply(remove_dollar)
df_4_new.loc[:, 'price'] = df_4_new['price'].apply(remove_dollar)

In [ ]:
type(df_1_new['price'][0])

In [ ]:
import requests
import json
import time

GROQ_API_KEY = "groq_api_key"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
MODEL_NAME = "llama-3.3-70b-versatile"

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:
def llm_product_match(prod1, prod2):
    system_prompt = """You are an expert product comparator. 
    Compare two pool-related products and decide if they are the same based on title, overview, and description even if brand differs. 
    Reply only 'Yes' or 'No'."""

    user_prompt = f"""
Product 1:
Title: {prod1.get('title', '')}
Overview: {prod1.get('overview', '')}
Description: {prod1.get('description', '')}

Product 2:
Title: {prod2.get('title', '')}
Overview: {prod2.get('overview', '')}
Description: {prod2.get('description', '')}

Are these products essentially the same? (Yes/No)
"""

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 10
    }

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(GROQ_URL, headers=headers, data=json.dumps(payload))

    if response.status_code != 200:
        print(f"⚠ LLM API Error {response.status_code}: {response.text}")
        return False

    try:
        result = response.json()
        reply = result['choices'][0]['message']['content'].strip().lower()
        return reply.startswith('yes')
    except Exception as e:
        print(f"⚠ JSON Parse Error: {e}, Raw Response: {response.text}")
        return False


In [ ]:
def adjust_price_logic(your_price, competitor_price_list):
    """
    Rules:
    - If competitors all have lower prices, set your price 1% below lowest competitor
    - If your price already best, keep it
    - If mixed, adjust to average of lowest two competitor prices minus 1 unit
    """
    valid_prices = [p for p in competitor_price_list if p > 0]

    if not valid_prices:
        return your_price  # No valid competitor price to compare

    min_price = min(valid_prices)
    sorted_prices = sorted(valid_prices)

    # If all competitors cheaper
    if all(p < your_price for p in valid_prices):
        return round(min_price - 1, 2)

    # If mixed, average lowest two and adjust
    if len(sorted_prices) >= 2:
        avg = (sorted_prices[0] + sorted_prices[1]) / 2
        return round(avg - 1, 2)

    # Else keep your price
    return your_price


In [ ]:
df_2_new = df_2_new.copy()
df_2_new['new_price'] = df_2_new['price']
df_2_new['competitor_prices'] = [{} for _ in range(len(df_2_new))]


for idx, row in df_2_new.iterrows():
    
    competitor_price_dict = {}
    matched_prices = []

    for df, comp_name in zip([df_1_new, df_3_new, df_4_new], ["Competitor_1", "Competitor_3", "Competitor_4"]):
        
        for _, comp_row in df.iterrows():
            
            if llm_product_match(row, comp_row):
                price = comp_row.get('price', 0)
                competitor_price_dict[comp_name] = price
                matched_prices.append(price)

                print(f"✅ Matched Product:")
                print(f"→ Site Product: {row['title']} | {row['link']}")
                print(f"→ Competitor: {comp_name} | {comp_row['title']} | {comp_row['link']}")
                print("-" * 80)
            time.sleep(2)
    df_2_new.at[idx, 'competitor_prices'] = competitor_price_dict
    df_2_new.at[idx, 'new_price'] = adjust_price_logic(row['price'], matched_prices)


In [ ]:
df_2_new